In [1]:
# import torch
import pywt
import numpy as np
import matplotlib.pyplot as plt
scale_min = 1
scale_max = 201
dscale = 0.5
FIN_WIDTH = 400
TOLERACE = 400
pad_to_for_planck_window = 90
p2 = pad_to_for_planck_window

def noisewt(postmerger, sam_p = 1, getfreqs=False):
    sam_f = 1 / sam_p
    scales = np.arange(scale_min, scale_max, dscale)

    # CWT on the gwf using the Morlet wavelet
    coefs, freqs = pywt.cwt(postmerger, scales, "morl", sampling_period=sam_p)

    # Normalising the coefficient matrix using the Frobenius norm
    Z = (np.abs(coefs)) / (np.linalg.norm(coefs))
    if getfreqs:
        return Z, freqs
    return Z

In [4]:
def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real

In [16]:
import numpy as np


class NoiseGenerator:
    """Generates gaussian noise with a specified power spectral density.
    Noise is generated by first creating a gaussian white base noise in the
    frequency domain, and then shaping it using a colouring function. Various
    colouring functions are provided for convenience.
    Examples
    --------
    >>> ng = NoiseGenerator()
    Generate 100000 white noise points sampled at 1 kHz with a PSD of 0.1.
    >>> white = ng.generate(1e-3, 100000, colour=ng.white(0.1))
    Generate some other colours of noise at a sampling rate of 1 MHz.
    >>> pink = ng.generate(1e-6, 1000000, colour=ng.pink(10.0))
    >>> blue = ng.generate(1e-6, 1000000, colour=ng.blue())
    Use a custom piecewise colouring function to generate specific noise.
    >>> frequencies = [90.0, 100.0, 110.0, 450.0, 500.0, 550.0]
    >>> psds = [0.1, 10.0, 0.01, 0.01, 2.0, 0.001]
    >>> colour = ng.piecewise_logarithmic(frequencies, psds)
    >>> custom = ng.generate(1e-4, 1000000, colour=colour)
    """

    rng = np.random.default_rng()

    def generate(self, dt, n, colour=None):
        """Generates uniformly sampled noise of a particular colour.
        Parameters
        ----------
        dt : float
            Sampling period, in seconds.
        n : int
            Number of samples to generate.
        colour : function, optional
            Colouring function that specifies the PSD at a given frequency. If
            not specified, the noise returned will be white Gaussian noise with
            a PSD of 1.0 across the entire frequency range.
        Returns
        -------
        numpy.array
            Length `n` array of sampled noise.
        """

        f, x_f = self._base_noise(dt, n)

        if colour:
            x_f *= np.sqrt(colour(f))

        return np.fft.irfft(x_f)

    @staticmethod
    def white(scale=1.0):
        """Creates a white noise colouring function.
        Parameters
        ----------
        scale : float, optional
            Multiplier to adjust the scale of the white noise.
        Returns
        -------
        function
            White noise colouring function that can be used with `generate()`.
            The function returned will be :math:`y(f) = s`, where :math:`s` is
            `scale`.
        """

        return lambda f: scale

    @staticmethod
    def pink(scale=1.0):
        """Creates a pink noise colouring function.
        Parameters
        ----------
        scale : float, optional
            Multiplier to adjust the scale of the pink noise.
        Returns
        -------
        function
            Pink noise colouring function that can be used with `generate()`.
            The function returned will be :math:`y(f) = s / f`, where :math:`s`
            is `scale`. At f = 0, the function will simply return 0.0.
        """

        return lambda f: scale / np.where(f == 0.0, np.inf, f)

    @staticmethod
    def brownian(scale=1.0):
        """Creates a brownian noise colouring function.
        Parameters
        ----------
        scale : float, optional
            Multiplier to adjust the scale of the brownian noise.
        Returns
        -------
        function
            Brownian noise colouring function that can be used with
            `generate()`. The function returned will be
            :math:`y(f) = s / f ^ 2`, where :math:`s` is `scale`. At f = 0, the
            function will simply return 0.0.
        """

        return lambda f: scale / np.where(f == 0.0, np.inf, f**2)

    @staticmethod
    def blue(scale=1.0):
        """Creates a blue noise colouring function.
        Parameters
        ----------
        scale : float, optional
            Multiplier to adjust the scale of the blue noise.
        Returns
        -------
        function
            Blue noise colouring function that can be used with `generate()`.
            The function returned will be :math:`y(f) = s * f`, where :math:`s`
            is `scale`.
        """

        return lambda f: scale * f

    @staticmethod
    def violet(scale=1.0):
        """Creates a blue noise colouring function.
        Parameters
        ----------
        scale : float, optional
            Multiplier to adjust the scale of the violet noise.
        Returns
        -------
        function
            Violet noise colouring function that can be used with `generate()`.
            The function returned will be :math:`y(f) = s * f ^ 2`, where
            :math:`s` is `scale`.
        """

        return lambda f: scale * f**2

    # Some common aliases for the various colours of noise
    brown = brownian
    red = brownian
    azure = blue
    purple = violet

    @staticmethod
    def piecewise_logarithmic(frequencies, psds):
        """Creates a custom piecewise colouring function
        Parameters
        ----------
        frequencies : numpy.array
            Array of frequencies, in Hz
        psds : numpy.array
            Array of PSDs
        Returns
        -------
        function
            Custom noise colouring function that can be used with `generate()`.
            The function is linearly interpolated in log space for the given
            frequencies and PSDs. Values outside the range of frequencies given
            are set to the PSD of the closest endpoint.
        """

        # Convert to log space
        log_frequencies = np.log(frequencies)
        log_psds = np.log(psds)

        # Create a closure for our colour function that suppresses the warning
        # about np.log(0) (which correctly returns -np.inf anyway)
        def colour(f):
            with np.errstate(divide="ignore"):
                return np.exp(np.interp(np.log(f), log_frequencies, log_psds))

        return colour

    def _base_noise(self, dt, n):
        """Produces a frequency domain representation of uniformly sampled
        Gaussian white noise.
        """

        # Calculate random frequency components
        f = np.fft.rfftfreq(n, dt)
        x_f = self.rng.normal(0,
            0.5, len(f)) + 1j * self.rng.normal(0, 0.5, len(f))
        x_f *= np.sqrt(n / dt)

        # Ensure our 0 Hz and Nyquist components are purely real
        x_f[0] = np.abs(x_f[0])
        if len(f) % 2 == 0:
            x_f[-1] = np.abs(x_f[-1])

        return f, x_f

In [5]:
psd = np.loadtxt("aLIGO_O4_high_asd.txt")
plt.loglog(psd[:,0], psd[:,1], label='PSD')

In [25]:
ng = NoiseGenerator()
colour = ng.piecewise_logarithmic(psd[:,0],psd[:,1])


In [35]:
%%timeit -n 10
custom = ng.generate(1e-4, 400, colour=colour)

56.9 µs ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
%%timeit -n 16 -r 1
custom = ng.generate(1e-4, 400, colour=colour)
noisewt(custom)

44.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 16 loops each)


In [42]:
custom = ng.generate(1e-4, 10000, colour=colour)

In [46]:
np.fft.rfft(custom)

array([ 6.89614554e-07+0.00000000e+00j, -4.56078030e-08-3.05986187e-07j,
        9.82926803e-09-6.92208632e-08j, ...,
       -1.64225321e-08-4.82449697e-09j, -3.55311512e-08+5.14964630e-08j,
        1.14632762e-08+0.00000000e+00j])

In [48]:
custom = ng.generate(1e-4, 400, colour=colour)
len(custom)

400